This notebook contains my practice notes and results of fast.ai's chaptar 2 of linear algebra: http://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/2.%20Topic%20Modeling%20with%20NMF%20and%20SVD.ipynb

In [29]:
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install matplotlib

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### setting up

In [30]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plot

%matplotlib inline 
np.set_printoptions(suppress=True)

### set train and test data

In [31]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

### Get some glimps of data

In [32]:
newsgroups_train.filenames.shape, newsgroups_train.target.shape

((2034,), (2034,))

### extract word frequency matrix (bag of words)

In [33]:
# using sklearn to extract frequency matrix
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
# specify the stopwords rule to be used when creating an instance of CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
# see newsgroups train data
print(newsgroups_train.data[0], len(newsgroups_train.data))
# create documents X words Matrix (bag of words Matrix)
vectors = vectorizer.fit_transform(newsgroups_train.data)
print(vectors) # its numpy compressed sparse metrix into a single row
# convert the sparse matrix to dense matrix
vectors = vectors.todense()
print(vectors, vectors.shape)

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych 2034
  (0, 21025)	1
  (0, 3998)	1
  (0, 5546)	1
  (0, 10605)	2
  (0, 20973)	1
  (0, 19841)	1
  (0, 2408)	1
  (0, 14706)	1
  (0, 20977)	1
  (0, 23828)	2
  (0, 21208)	1
  (0, 15301)	1
  (0, 21084)	1
  (0, 9848)	1
  (0, 22878)	1
  (0, 13023)	2
  (0, 14154)	1
  (0, 8554)	2
  (0, 18949)	1
  (0, 18704)	1
  (0, 19066)	3
  (0, 17464)	2
  (0, 18699)	1
  (0, 7698)	1
  (0, 11203)	1
  :	:
  (2032, 58

In [35]:
# get vocabulary dictionary
vocab = np.array(vectorizer.get_feature_names())
print(vocab, '\n', vocab.shape, '\n', vocab[7000:7020])

['00' '000' '0000' ... 'zware' 'zwarte' 'zyxel'] 
 (26576,) 
 ['cosmonauts' 'cosmos' 'cosponsored' 'cost' 'costa' 'costar' 'costing'
 'costly' 'costruction' 'costs' 'cosy' 'cote' 'couched' 'couldn' 'council'
 'councils' 'counsel' 'counselees' 'counselor' 'count']


### Singular Value Decomposition (SVD)

**Idea:** In this document classification problem, we assume each document is can be classified into any of the categories provided purely based on the words used in it. Each distinguishing word in a document from another contributes to the determination of class label to which it belongs.

In Linear Algebra perspective, the features of each topic to be orthonormal. SVD helps us to factorize a single matrix into three matrices - one matrix with orthogonal columns, one with orthogonal rows and another diagonal matrix with relative importance of each factor.